In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
from load_data import load_data
import cv2 


dirname = os.path.dirname(os.path.abspath('Ex1'))
data_path = os.path.join(dirname,'data', 'demo.p')
data = load_data(data_path)

# extract data
#point cloud
xyz_velodyne = data['velodyne'][:,0:3] #data from 0 to 3-1

#projection matrices
P = data['P_rect_20']
T = data['T_cam2_velo']
#image
image2 = data['image_2']




laser_label = np.zeros(64)
color_map = {1: [0, 0, 255], 2:[255,0,0], 3:[0,255,0], 4:[30, 30, 255]}#division par 4 avec reste


#cône (360 deg)avec 0,4 deg de différences d'ouverture
#distance au cone le plus proche en x,y,z (sphère autour du point?)

#filter points with negative x
velodyne_fltrd = []
laser_label_fltrd = []
for i in range(xyz_velodyne.shape[0]):
    if xyz_velodyne[i, 0] >= 0:
        velodyne_fltrd.append(xyz_velodyne[i, :])
        sem_label_fltrd.append(sem_label[i])
velodyne_fltrd = np.array(velodyne_fltrd)
slaser_label_fltrd = np.array(laser_label_fltrd)

#Projection of point cloud in image 2 coordinates
a = np.ones((velodyne_fltrd.shape[0],1))
velodyne_fltrd = np.hstack((velodyne_fltrd, a))
velodyne_fltrd = np.transpose(velodyne_fltrd)

extrin_calib = np.matmul(T,velodyne_fltrd)
proj_cloud = np.matmul(P,extrin_calib)/extrin_calib[2,:] #normalization by Zc

u,v,k = proj_cloud   #k is an array of ones

u = u.astype(np.int32)
v = v.astype(np.int32)



#Draw lasser ID color of the point cloud on image
img = image2.astype(np.uint8)
color=np.zeros(velodyne_fltrd.shape[1])
for i in range(velodyne_fltrd.shape[1]):
    label=laser_label_fltrd[i][0]
    color = color_map.get(label)
    # Draw a circle of corresponding color 
    cv2.circle(img,(u[i],v[i]), 1, color, -1)
cv2.imshow('image2',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

{0: [0, 0, 0], 1: [0, 0, 255], 10: [245, 150, 100], 11: [245, 230, 100], 13: [250, 80, 100], 15: [150, 60, 30], 16: [255, 0, 0], 18: [180, 30, 80], 20: [255, 0, 0], 30: [30, 30, 255], 31: [200, 40, 255], 32: [90, 30, 150], 40: [255, 0, 255], 44: [255, 150, 255], 48: [75, 0, 75], 49: [75, 0, 175], 50: [0, 200, 255], 51: [50, 120, 255], 52: [0, 150, 255], 60: [170, 255, 150], 70: [0, 175, 0], 71: [0, 60, 135], 72: [80, 240, 150], 80: [150, 240, 255], 81: [0, 0, 255], 99: [255, 255, 50], 252: [245, 150, 100], 256: [255, 0, 0], 253: [200, 40, 255], 254: [30, 30, 255], 255: [90, 30, 150], 257: [250, 80, 100], 258: [180, 30, 80], 259: [255, 0, 0]}
